In [1]:
import pandas as pd

df = pd.read_csv('CosciaRios2012_DataBase.csv')
df.head()

,Code,State,Year,Beltran_Leyva,Beltran_Leyva_Family,Familia,Golfo,Juarez,Sinaloa,Sinaloa_Family,Tijuana,Zetas,Otros
0,1001,1,1990,0,0,0,0,0,0,0,0,0,0
1,1001,1,1991,0,0,0,0,0,0,0,0,0,0
2,1001,1,1992,0,0,0,0,0,0,0,0,0,0
3,1001,1,1993,0,0,0,0,0,0,0,0,0,0
4,1001,1,1994,0,0,0,0,0,0,0,0,0,0


In [3]:
# Count rows where all columns from 'Beltran_Leyva' onwards are 0s
beltran_start = df.columns.get_loc('Beltran_Leyva')
zero_rows = (df.iloc[:, beltran_start:] == 0).all(axis=1).sum()
zero_rows

48138

In [5]:
# Find the max year
max_year = df['Year'].max()

# Select rows for the max year
max_year_df = df[df['Year'] == max_year]

# Check if columns 4 to 13 (by position) are all 0s for each row
empty_codes = (max_year_df.iloc[:, 3:13] == 0).all(axis=1)

# Count empty codes and total codes in max year
num_empty_codes = empty_codes.sum()
total_codes = len(max_year_df)

print(f"Empty codes in {max_year} year: {num_empty_codes}")
print(f"Total codes in {max_year} year: {total_codes}")

Empty codes in 2010 year: 1747
Total codes in 2010 year: 2456


In [6]:
max_year_df.head()

,Code,State,Year,Beltran_Leyva,Beltran_Leyva_Family,Familia,Golfo,Juarez,Sinaloa,Sinaloa_Family,Tijuana,Zetas,Otros
20,1001,1,2010,0,0,1,0,1,0,0,0,1,0
41,1002,1,2010,0,0,1,0,0,1,0,0,0,0
62,1003,1,2010,0,0,0,0,0,0,0,0,1,0
83,1004,1,2010,0,0,0,0,0,0,0,0,0,0
104,1005,1,2010,0,0,0,0,0,0,0,0,1,0


In [8]:
# Count how many times each family is present in each municipality, collapsing all years
# 'Code' is municipality, families are columns from 'Beltran_Leyva' onwards
family_start = df.columns.get_loc('Beltran_Leyva')
family_cols = df.columns[family_start:]

# Melt the dataframe to long format for counting
melted = df.melt(id_vars=['Code'], value_vars=family_cols, var_name='Family', value_name='Presence')

# Count presence (assuming >0 means present)
presence_counts = melted[melted['Presence'] > 0].groupby(['Code', 'Family']).size().reset_index(name='Count')
presence_counts.head()

,Code,Family,Count
0,1001,Familia,5
1,1001,Golfo,1
2,1001,Juarez,3
3,1001,Sinaloa,3
4,1001,Tijuana,2


In [9]:
# For each column from column 4 onward, count number of times the column has 1 in each code through all years
start_col = 3  # column index 4 (0-based)
family_cols = df.columns[start_col:]

# Count number of times each family column has 1 for each code
counts = {}
for col in family_cols:
    counts[col] = df[df[col] == 1].groupby('Code').size()

# Convert to DataFrame for easier viewing
counts_df = pd.DataFrame(counts).fillna(0).astype(int)
counts_df.head()

,Beltran_Leyva,Beltran_Leyva_Family,Familia,Golfo,Juarez,Sinaloa,Sinaloa_Family,Tijuana,Zetas,Otros
Code,,,,,,,,,,
1001,0,0,5,1,3,3,0,2,5,0
1002,0,0,1,0,3,2,0,0,4,0
1003,0,0,0,3,0,2,0,0,3,0
1005,0,0,0,2,0,0,0,0,3,0
1006,0,0,0,0,0,0,0,0,3,0


In [10]:
# Count number of unique codes (municipalities)
num_unique_codes = df['Code'].nunique()
print(f"Number of unique codes: {num_unique_codes}")

Number of unique codes: 2457


In [12]:
# Count number of columns from 4th column onward and show their names
start_col = 3  # column index 4 (0-based)
family_cols = df.columns[start_col:]
num_family_cols = len(family_cols)
print(f"Number of columns from 4th onward: {num_family_cols}")
print("Column names:", list(family_cols))

Number of columns from 4th onward: 10
Column names: ['Beltran_Leyva', 'Beltran_Leyva_Family', 'Familia', 'Golfo', 'Juarez', 'Sinaloa', 'Sinaloa_Family', 'Tijuana', 'Zetas', 'Otros']


In [13]:
# Count number of columns in counts_df that have only zeros
num_zero_columns = (counts_df.sum(axis=0) == 0).sum()
print(f"Number of columns with only zeros in counts_df: {num_zero_columns}")

Number of columns with only zeros in counts_df: 0


In [14]:
# Calculate the sum of each column in counts_df
column_sums = counts_df.sum(axis=0)
print(column_sums)

Beltran_Leyva            588
Beltran_Leyva_Family     154
Familia                  644
Golfo                   1320
Juarez                   421
Sinaloa                  874
Sinaloa_Family           153
Tijuana                  316
Zetas                   1587
Otros                    171
dtype: int64


In [15]:
# Count number of codes for each column in counts_df that are not zero
nonzero_code_counts = (counts_df != 0).sum(axis=0)
print(nonzero_code_counts)

Beltran_Leyva           282
Beltran_Leyva_Family     86
Familia                 308
Golfo                   438
Juarez                  151
Sinaloa                 307
Sinaloa_Family           78
Tijuana                 115
Zetas                   585
Otros                    86
dtype: int64
